# Minimal working example of the circleFitter code
This is a minimal working example, this imports the code, sets up a data set and relevant config info (November 2021 AuxTel run)

In [20]:
import circleFitter as fc
import numpy as np
from scipy import linalg

In [21]:
dayObs = 20211102
s1 = [s for s in range(501, 554+1)]
seqList=s1
config = {"Halfbox": 1200, "kernel": 61, "minclip": 0.2, "maxclip" :0.8,
                  "outerRadius": 750, "innerRadius": 290, "vmin": 10,
                  "vmax": 500, "normPercent": 90, "skyPercent": 15,
                  "minDistOuter": 200, "minDistInner": 300
                  }

In [22]:

efd_infos, dxs, dys, planeSkew = fc.findCircles(dayObs, seqList, planeSkew = True, config = config)

Seq_num: 501, dx_offset=-18, dy_offset=-6
Seq_num: 502, dx_offset=-22, dy_offset=-10
Seq_num: 503, dx_offset=-20, dy_offset=2
Seq_num: 504, dx_offset=-16, dy_offset=-2
Seq_num: 505, dx_offset=-14, dy_offset=-16
Seq_num: 506, dx_offset=-2, dy_offset=-6
Seq_num: 507, dx_offset=4, dy_offset=0
Seq_num: 508, dx_offset=4, dy_offset=-4
Seq_num: 509, dx_offset=0, dy_offset=-2
Seq_num: 510, dx_offset=4, dy_offset=0
Seq_num: 511, dx_offset=4, dy_offset=8
Seq_num: 512, dx_offset=12, dy_offset=8
Seq_num: 513, dx_offset=12, dy_offset=0
Seq_num: 514, dx_offset=12, dy_offset=4
Seq_num: 515, dx_offset=16, dy_offset=2
Seq_num: 516, dx_offset=24, dy_offset=6
Seq_num: 517, dx_offset=32, dy_offset=6
Seq_num: 518, dx_offset=28, dy_offset=14
Seq_num: 519, dx_offset=-8, dy_offset=16
Seq_num: 520, dx_offset=-6, dy_offset=26
Seq_num: 521, dx_offset=2, dy_offset=14
Seq_num: 522, dx_offset=-6, dy_offset=12
Seq_num: 523, dx_offset=-4, dy_offset=8
Seq_num: 524, dx_offset=-2, dy_offset=20
Seq_num: 525, dx_offset=-4

We have now done all the circle fitting, but what does our results actually tell us?

In [23]:
x = [d['x'] for d in efd_infos]
y = [d['y'] for d in efd_infos]
grad_x = [d[0] for d in planeSkew]
grad_y = [d[1] for d in planeSkew]

# we need a row of ones for the fitting we are about it
ones = np.ones_like(x)
A = np.c_[x,y, ones]

### Now we will do two fits
one for the the displacements, and one for the planeSkew. First the displacements

In [25]:
C_1,_,_,_ =linalg.lstsq(A,dxs)
C_2,_,_,_ =linalg.lstsq(A,dys)

Anew = np.array([[C_1[0],C_1[1]],[C_2[0],C_2[1]]])
offset = np.array([[C_1[2]],[C_2[2]]])

center = linalg.inv(Anew)@(-offset)
print(f"the center according to displacements: {center}")

the center according to displacements: [[-3.67968985]
 [ 2.35730859]]


In [26]:
C_3,_,_,_ =linalg.lstsq(A,grad_x)
C_4,_,_,_ =linalg.lstsq(A,grad_y)

Bnew = np.array([[C_3[0],C_3[1]],[C_4[0],C_4[1]]])
Boffset = np.array([[C_3[2]],[C_4[2]]])
center2 = linalg.inv(Bnew)@(-Boffset)
print(f"the center according to the planeSkew: {center2}")

the center according to the planeSkew: [[-3.92733066]
 [ 2.24245547]]
